In [ ]:
import pandas as pd
import os
import re
from typing import List, Dict
from IPython.display import displayimport os

In [2]:
def convert_stata_file(input_path, output_format='csv', output_path=None):
    """
    Convert Stata .dta file to CSV or PKL format
    
    Parameters:
    -----------
    input_path : str
        Path to the input .dta file
    output_format : str
        Desired output format ('csv' or 'pkl')
    output_path : str, optional
        Path for the output file. If None, will use the same name as input
        
    Returns:
    --------
    str
        Path to the converted file
    """
    # Validate input file exists
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"Input file not found: {input_path}")
        
    # Validate output format
    if output_format.lower() not in ['csv', 'pkl']:
        raise ValueError("Output format must be 'csv' or 'pkl'")
    
    # Generate output path if not provided
    if output_path is None:
        base_path = os.path.splitext(input_path)[0]
        output_path = f"{base_path}.{output_format.lower()}"
    
    try:
        # Read the Stata file
        print(f"Reading Stata file: {input_path}")
        df = pd.read_stata(input_path)
        
        # Convert to specified format
        if output_format.lower() == 'csv':
            print(f"Converting to CSV: {output_path}")
            df.to_csv(output_path, index=False)
        else:
            print(f"Converting to pickle: {output_path}")
            df.to_pickle(output_path)
            
        print(f"Conversion complete. File saved as: {output_path}")
        return output_path
        
    except Exception as e:
        print(f"Error during conversion: {str(e)}")
        raise
    

Reading Stata file: /Users/michaelzhu/Desktop/Stanford/CS229/randhrs1992_2020v2_STATA/randhrs1992_2020v2.dta
Converting to pickle: /Users/michaelzhu/Desktop/Stanford/CS229/randhrs1992_2020v2_STATA/randhrs1992_2020v2.pkl
Conversion complete. File saved as: /Users/michaelzhu/Desktop/Stanford/CS229/randhrs1992_2020v2_STATA/randhrs1992_2020v2.pkl


In [ ]:
convert_stata_file(
    input_path="/Users/michaelzhu/Desktop/Stanford/CS229/randhrs1992_2020v2_STATA/randhrs1992_2020v2.dta",
    output_format="pkl",
    output_path="/Users/michaelzhu/Desktop/Stanford/CS229/randhrs1992_2020v2_STATA/randhrs1992_2020v2.pkl"
)

In [3]:
df = pd.read_pickle("/Users/michaelzhu/Desktop/Stanford/CS229/randhrs1992_2020v2_STATA/randhrs1992_2020v2.pkl")
print(f"{len(df)} rows and {len(df.columns)} columns")

In [6]:
def get_selected_vars(df: pd.DataFrame, wave: int, prefix: str) -> List[str]:
    """
    Get variables that match specific patterns for a given wave and prefix.
    """
    selected_vars = ['hhidpn', f'inw{wave}']
    
    # Get all column names
    all_vars = df.columns.tolist()
    
    # Regular expressions for matching
    wave_pattern = f"{prefix}{wave}[A-Za-z]"
    always_pattern = f"^{prefix}a[A-Za-z]"
    
    # Add matching variables
    for var in all_vars:
        if re.search(wave_pattern, var) or re.search(always_pattern, var):
            selected_vars.append(var)
            
    return selected_vars

def process_wave_data(df: pd.DataFrame, wave: int, prefix: str) -> pd.DataFrame:
    """
    Process data for a specific wave and prefix.
    """
    # Get relevant variables
    selected_vars = get_selected_vars(df, wave, prefix)
    
    # Select variables and filter rows
    wave_df = df[selected_vars].copy()
    wave_df = wave_df[wave_df[f'inw{wave}'] == 1].copy()
    
    # Add wave column
    wave_df['wave'] = wave
    
    # Rename columns - remove wave number from variable names
    rename_dict = {}
    for var in selected_vars:
        if prefix == 's' and var in [f's{wave}tr20', f's{wave}tr40']:
            # Special case for tr20 and tr40
            new_name = var.replace(f's{wave}tr', f's{wave}rtr')
            rename_dict[var] = new_name
            
        if var != 'hhidpn' and var != f'inw{wave}':
            new_name = var.replace(f'{prefix}{wave}', prefix)
            rename_dict[var] = new_name
    
    wave_df = wave_df.rename(columns=rename_dict)
    
    # Drop inw column
    if prefix in ['r', 'h']:
        wave_df = wave_df.drop(columns=[f'inw{wave}'])
        
    return wave_df
    

In [17]:
import pandas as pd
import re
import os
from pathlib import Path

# Configure pandas display options
pd.set_option('display.max_columns', None)

def get_selected_vars(data_dict, wave_num, prefix):
    """
    Get selected variables based on wave number and prefix pattern
    Similar to Stata's describe and regex matching functionality
    
    Args:
        data_dict: Dictionary or similar containing variable information
        wave_num: Wave number (1-15)
        prefix: Prefix to match ('s', 'r', or 'h')
    """
    selected_vars = ['hhidpn', f'inw{wave_num}']
    
    for var in data_dict.columns:
        # Match wave-specific variables
        if re.match(f'{prefix}{wave_num}[A-Za-z]', var):
            selected_vars.append(var)
        # Match common variables across waves
        if re.match(f'^{prefix}a[A-Za-z]', var):
            selected_vars.append(var)
            
    return selected_vars

def process_wave_data(data, wave_num, prefix):
    """
    Process data for a specific wave and prefix
    
    Args:
        data: pandas DataFrame containing the HRS data
        wave_num: Wave number (1-15)
        prefix: Prefix to match ('s', 'r', or 'h')
    """
    # Get selected variables
    selected_vars = get_selected_vars(data, wave_num, prefix)
    
    # Subset data
    wave_data = data[selected_vars].copy()
    wave_data = wave_data[wave_data[f'inw{wave_num}'] != 0].copy()
    
    # Add wave indicator
    wave_data['wave'] = wave_num
    
    # Special handling for 's' prefix
    if prefix == 's':
        # Handle special cases for tr20 and tr40
        selected_vars = [v.replace('tr20', 'rtr20').replace('tr40', 'rtr40') 
                        for v in selected_vars]
    
    # Rename variables to remove wave numbers
    rename_dict = {old: new.replace(f'{prefix}{wave_num}', prefix) 
                  for old, new in zip(selected_vars, selected_vars)}
    wave_data = wave_data.rename(columns=rename_dict)
    
    # Drop inw column if not 's' prefix
    if prefix != 's':
        wave_data = wave_data.drop(columns=[f'inw{wave_num}'])
        
    return wave_data

/var/folders/f8/8tg6b2p95rs0b9f1frtgrxvh0000gn/T/ipykernel_33408/1822511503.py:86: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_data[prefix] = pd.concat(prefix_data, axis=0, ignore_index=True)
/var/folders/f8/8tg6b2p95rs0b9f1frtgrxvh0000gn/T/ipykernel_33408/1822511503.py:86: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_data[prefix] = pd.concat(prefix_data, axis=0, ignore_index=True)


In [ ]:
# Read the original data
hrs_data = df

# Process each prefix type
prefixes = ['s', 'r', 'h']
combined_data = {}

for prefix in prefixes:
    # Process each wave
    prefix_data = []
    for wave in range(1, 16):  # 1 to 15
        wave_data = process_wave_data(hrs_data, wave, prefix)
        prefix_data.append(wave_data)
    
    # Combine all waves for this prefix
    combined_data[prefix] = pd.concat(prefix_data, axis=0, ignore_index=True)

# Merge all prefix datasets
final_data = combined_data['s']
for prefix in ['r', 'h']:
    final_data = pd.merge(
        final_data, 
        combined_data[prefix],
        on=['hhidpn', 'wave'],
        how='inner',
        validate='1:1'
    )

# Save the final dataset
final_data.to_pickle('randhrs1992_2020v2_long_compact.pkl')

In [ ]:
final_data